In [ ]:
import ast


class SecurePythonExecutor:
    def __init__(self, allowed_imports=None, max_operations=1000):
        self.allowed_imports = set(allowed_imports or [])
        self.max_operations = max_operations
        self.operation_count = 0

    def _check_imports(self, node):
        if isinstance(node, ast.Import):
            for alias in node.names:
                if alias.name not in self.allowed_imports:
                    raise ImportError(f"Import of '{alias.name}' is not allowed.")
        elif isinstance(node, ast.ImportFrom):
            if node.module not in self.allowed_imports:
                raise ImportError(f"Import from '{node.module}' is not allowed.")

    def _count_operations(self, node):
        self.operation_count += 1
        if self.operation_count > self.max_operations:
            raise RuntimeError("Operation limit exceeded.")

    def _validate_node(self, node):
        for child in ast.iter_child_nodes(node):
            self._check_imports(child)
            self._count_operations(child)
            self._validate_node(child)

    def execute(self, code_str, exec_globals=None, exec_locals=None):
        self.operation_count = 0
        parsed_ast = ast.parse(code_str, mode='exec')
        self._validate_node(parsed_ast)
        exec_globals = exec_globals or {}
        exec_locals = exec_locals or {}
        exec(compile(parsed_ast, filename="<ast>", mode="exec"), exec_globals, exec_locals)
        return exec_locals

In [ ]:
executor = SecurePythonExecutor(allowed_imports=['math'], max_operations=100)
code = """
import math
result = math.sqrt(16)
"""
locals_after_exec = executor.execute(code)
print(locals_after_exec['result'])  # Output: 4.0

In [ ]:
code = """
import math
import numpy as np
result = math.sqrt(16)
"""
parsed_ast = ast.parse(code, mode='exec')
for child in ast.iter_child_nodes(parsed_ast):
    print(child)

In [ ]:
from smolagents.local_python_executor import LocalPythonExecutor

# Set up custom executor, authorize package "numpy"
custom_executor = LocalPythonExecutor(["numpy"])

# Utilisty for pretty printing errors


def run_capture_exception(command: str):
    try:
        return custom_executor(harmful_command)
    except Exception as e:
        print("ERROR:\n", e)


# Undefined command just do not work
harmful_command="import numpy as np; output = np.array([1, 2, 3])"
run_capture_exception(harmful_command)